In [1]:
import pandas as pd 
import streamlit as st
from functools import reduce

# Data Preprocessing

1. The data contains missing values. For all three CSVs, use forward filling for imputing the missing data. Forward filling uses earlier values of the country for filling later ones (tip: there exist a Pandas function for that)

In [2]:
life = pd.read_csv('C:/Users/deniz/OneDrive/Dokumente/GitHub/streamlit/life_expectancy_years.csv').fillna(method='ffill', axis=1)
pop = pd.read_csv('C:/Users/deniz/OneDrive/Dokumente/GitHub/streamlit/population_total.csv').fillna(method='ffill', axis=1)
gni = pd.read_csv('C:/Users/deniz/OneDrive/Dokumente/GitHub/streamlit/gnipercapita_ppp_current_international.csv').fillna(method='ffill', axis=1)
gni = gni.fillna(method='bfill', axis=0)

In [5]:
gni.isna().sum()

country    0
1990       0
1991       0
1992       0
1993       0
1994       0
1995       0
1996       0
1997       0
1998       0
1999       0
2000       0
2001       0
2002       0
2003       0
2004       0
2005       0
2006       0
2007       0
2008       0
2009       0
2010       0
2011       0
2012       0
2013       0
2014       0
2015       0
2016       0
2017       0
2018       0
2019       0
dtype: int64

2. For each CSV, transform the dataframe into a so called "tidy data format" (see e.g. https://en.wikipedia.org/wiki/Tidy_data). Each dataframe then should have only three columns: (1) country, (2) year, (3) the KPI (that means either a) life expectancy, b) population, or 3) GNI per capita)

In [10]:
formatted_life = pd.melt(life,id_vars=["country"], var_name="year", value_name="life")
formatted_life = formatted_life.sort_values(by=["country"])
formatted_life.head(10)

,country,year,life
0,Afghanistan,1800,28.2
53669,Afghanistan,2087,75.9
2618,Afghanistan,1814,28.1
15895,Afghanistan,1885,28.5
40392,Afghanistan,2016,61.2
28424,Afghanistan,1952,33.6
27863,Afghanistan,1949,32.4
48620,Afghanistan,2060,72.3
7667,Afghanistan,1841,27.8
23375,Afghanistan,1925,31.0


In [11]:
formatted_pop = pd.melt(pop,id_vars=["country"], var_name="year", value_name="pop")
formatted_pop = formatted_pop.sort_values(by=["country"])
formatted_pop.head(10)

,country,year,pop
0,Afghanistan,1800,3280000
53625,Afghanistan,2075,76200000
5070,Afghanistan,1826,3370000
56355,Afghanistan,2089,76800000
9165,Afghanistan,1847,3700000
49530,Afghanistan,2054,67400000
12090,Afghanistan,1862,3980000
24180,Afghanistan,1924,9200000
34515,Afghanistan,1977,13200000
29835,Afghanistan,1953,8040000


In [17]:
formatted_gni = pd.melt(gni,id_vars=["country"], var_name="year", value_name="GNI")
formatted_gni = formatted_gni.sort_values(by=["country"])
formatted_gni.head(10)

,country,year,GNI
0,Afghanistan,1990,Afghanistan
3348,Afghanistan,2008,Afghanistan
3162,Afghanistan,2007,Afghanistan
2976,Afghanistan,2006,Afghanistan
5022,Afghanistan,2017,2230.0
2790,Afghanistan,2005,Afghanistan
2604,Afghanistan,2004,Afghanistan
2418,Afghanistan,2003,Afghanistan
2232,Afghanistan,2002,Afghanistan
2046,Afghanistan,2001,Afghanistan


3. You have to merge all three CVS. Merge the three dataframe into one dataframe, that just has 5 columns (country, year and all three KPIs from the three dataframes)

In [16]:
data_frames = [formatted_life, formatted_pop, formatted_gni]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['country', 'year'], how='outer'), data_frames)
df_merged.GNI.value_counts()

11000.0        25
860.0          23
10200.0        23
Djibouti       23
South Sudan    21
               ..
5300.0          1
9140.0          1
58300.0         1
88700.0         1
5330.0          1
Name: GNI, Length: 1465, dtype: int64

In [22]:
df1 = formatted_life.merge(formatted_pop, how='outer', on=('country', 'year')).merge(formatted_gni, how='outer', on=('country', 'year'))
df1.GNI.fillna(method='ffill')
df1.GNI.fillna(method='bfill')
df1.head()

,country,year,life_exp,pop,GNI
0,Afghanistan,1800,28.2,3280000,NaN
1,Afghanistan,2087,75.9,76900000,NaN
2,Afghanistan,1814,28.1,3280000,NaN
3,Afghanistan,1885,28.5,4460000,NaN
4,Afghanistan,2016,61.2,35400000,2150.0


In [10]:
df_merged.to_csv('life_pop_gni.csv')

In [23]:
df1.GNI.value_counts()

11000.0        25
860.0          23
10200.0        23
Djibouti       23
South Sudan    21
               ..
5300.0          1
9140.0          1
58300.0         1
88700.0         1
5330.0          1
Name: GNI, Length: 1465, dtype: int64